<a href="https://colab.research.google.com/github/tanyaryabov/ML/blob/master/IMDB_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
tfds.list_builders()

In [ ]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews', split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

train_example_batch, train_labels_batch= next(iter(train_data.batch(10)))
train_labels_batch


<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
hub_layer(train_example_batch[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_data.shuffle(10000).batch(512),
         epochs=20,
         validation_data=validation_data.batch(512),
         verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 97ms/step - loss: 2.1499 - accuracy: 0.4993 - val_loss: 0.7991 - val_accuracy: 0.5085
Epoch 2/20
30/30 [==============================] - 3s 91ms/step - loss: 0.7806 - accuracy: 0.5228 - val_loss: 0.6803 - val_accuracy: 0.5847
Epoch 3/20
30/30 [==============================] - 3s 91ms/step - loss: 0.6502 - accuracy: 0.6191 - val_loss: 0.6073 - val_accuracy: 0.6701
Epoch 4/20
30/30 [==============================] - 3s 90ms/step - loss: 0.5880 - accuracy: 0.6894 - val_loss: 0.5697 - val_accuracy: 0.7069
Epoch 5/20
30/30 [==============================] - 3s 90ms/step - loss: 0.5544 - accuracy: 0.7174 - val_loss: 0.5398 - val_accuracy: 0.7371
Epoch 6/20
30/30 [==============================] - 3s 92ms/step - loss: 0.5128 - accuracy: 0.7576 - val_loss: 0.5125 - val_accuracy: 0.7570
Epoch 7/20
30/30 [==============================] - 3s 93ms/step - loss: 0.4856 - accuracy: 0.7834 - val_loss: 0.4868 - val_accuracy: 0.7766
Epoch 8/20
30

In [ ]:
model.predict(["This is the worst movie I have ever seen",
              "An excellent movie that I enjoyed a lot",
              "how can one make such a horrible movie? there is no story, acting direction everything is very poor"])

array([[0.1761826 ],
       [0.9958751 ],
       [0.05734587]], dtype=float32)